* Script credits: Joao Silva : https://www.linkedin.com/in/joaomiguelarch/

# Chat with your IFC file
1. Load IFC and convert to SQL
2. LLM call: Question2Query
3. Execute SQL query
4. Construct natural answer

### Libraries

In [ ]:
# Install module dependencies
!pip install ifcopenshell
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 808.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


## 1. IFC to SQL

### Load and filter IFC

* IFC Open Shell 101 : https://docs.ifcopenshell.org/ifcopenshell-python/code_examples.html

In [ ]:
import ifcopenshell
import ifcopenshell.util
import ifcopenshell.util.element
import pandas as pd

# Clone the project repo
!git clone https://github.com/Mistrymm7/chat_with_ifc_llm_workshop.git
# Load the IFC file
file_path = "/content/chat_with_ifc_llm_workshop/ifc-sample.ifc"
ifc_file = ifcopenshell.open(file_path)
# Filter for a few specified Element types only
class_names = ["IfcDoor", "IfcWindow"]

Cloning into 'chat_with_ifc_llm_workshop'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 28 (delta 6), reused 11 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 784.68 KiB | 7.93 MiB/s, done.
Resolving deltas: 100% (6/6), done.


### Convert to CSV format

In [ ]:
outuput_csv_name = "/content/ifc_as_csv.xlsx"

from rich import print

# Create CSV Tables
with pd.ExcelWriter(outuput_csv_name, engine='openpyxl') as writer:
    for class_name in class_names:
        # Go through the objects
        result_df = pd.DataFrame()
        objects = ifc_file.by_type(class_name)
        for object in objects:
            class_data = {}

            # Get Pset properties
            psets  = ifcopenshell.util.element.get_psets(object)
            for name, value in psets.items():
                if name.startswith("Pset_"):
                    name = name.replace("Pset_", "") # removing "Pset_" prefix
                if isinstance(value, dict):
                    for key, val in value.items():
                        if key == "id" or key == "Reference":  # removing pset id property, removing Reference (its always = ObjectType)
                            continue
                        hierarchical_key_name = (name + "_" + key).replace(" ", "")
                        class_data[hierarchical_key_name] = val
                else:
                    pass
            class_df =  pd.DataFrame(class_data, index=[0])
            result_df = pd.concat([result_df, class_df], ignore_index=True)
        if(result_df.empty):
            continue

        # Remove parameters that are more empty than filled (noisy BIM data)
        threshold = len(result_df) / 2
        columns_to_drop = result_df.columns[result_df.isnull().sum() > threshold]
        if not columns_to_drop.empty:
            print("Dropping columns:", list(columns_to_drop))
        result_df = result_df.drop(columns=columns_to_drop)

        #Export the class sheet
        result_df.to_excel(writer, sheet_name=class_name, index=False)

Dropping columns:
[
    'M_Single-Flush:InsideDoor:353615_PanelOperation',
    'M_Single-Flush:InsideDoor:353615_PanelPosition',
    'M_Door-Interior-Double-FullGlass-Wood:1500x2000mm:353875_PanelOperation',
    'M_Door-Interior-Double-FullGlass-Wood:1500x2000mm:353875_PanelPosition',
    'Construction_SwingAngle',
    'Dimensions_BottomRail',
    'Dimensions_PanelWidth',
    'Dimensions_RoughHeight',
    'Dimensions_RoughWidth',
    'Dimensions_Stiles',
    'Dimensions_TopRail',
    'MaterialsandFinishes_DoorHandleMaterial',
    'MaterialsandFinishes_PanelMaterial',
    'MaterialsandFinishes_TrimMaterial',
    'Custom_Pset_RoughWidth',
    'Graphics_ShowGrill'
]

### Convert to SQL Database

In [ ]:
import sqlite3

csv_file_path = '/content/ifc_as_csv.xlsx'
conn = sqlite3.connect('/content/ifcdatabase.db')
cursor = conn.cursor()

# Load the Excel file, get a dictionary of DataFrames, one per sheet
sheets_dict = pd.read_excel(csv_file_path, sheet_name=None)

# Iterate over each sheet in the Excel file
for sheet_name, df in sheets_dict.items():
    # Get column names and types from the DataFrame
    columns =df.columns.str.strip()
    types = df.dtypes
    # Create a SQL command to create a table dynamically
    column_defs = []
    for col, dtype in zip(columns, types):
        if pd.api.types.is_integer_dtype(dtype):
            col_type = 'INTEGER'
        elif pd.api.types.is_float_dtype(dtype):
            col_type = 'REAL'
        else:
            col_type = 'TEXT'
        column_defs.append(f'"{col}" {col_type}')

    column_defs_str = ', '.join(column_defs)
    create_table_sql = f'CREATE TABLE IF NOT EXISTS {sheet_name} ({column_defs_str})'

    # Execute the SQL command to create the table
    cursor.execute(create_table_sql)

    # Insert DataFrame into SQLite database
    df.to_sql(sheet_name, conn, if_exists='append', index=False)

    print(f"Data from sheet '{sheet_name}' inserted into table '{sheet_name}'.")

# Verify data
print("Verifying...")
for sheet_name in sheets_dict.keys():
    cursor.execute(f'SELECT * FROM {sheet_name}')
    rows = cursor.fetchall()
    print(f"\nData from table '{sheet_name}':")
    for row in rows:
        print(row)

# Close the connection
conn.close()

Data from sheet 'IfcDoor' inserted into table 'IfcDoor'.

Data from sheet 'IfcWindow' inserted into table 'IfcWindow'.

Verifying...

Data from table 'IfcDoor':

(
    'SWINGING',
    'NOTDEFINED',
    'Metal',
    'Schematic Type',
    3.7021,
    0.0,
    0.270116960643959,
    0.0,
    'Interior',
    'By host',
    2134,
    51,
    25.0,
    25.0,
    76.0,
    915,
    3.18957899999998,
    0.119856109999999,
    None,
    'Inside door: Assembly Code',
    None,
    'Inside door: Description',
    'Inside door: Fire Rating',
    'Inside door: Keynote',
    'Inside door: Manufacturer',
    'Inside door: Model',
    '23.30.10.00',
    'Doors',
    'Inside door: Type Comments',
    'Inside door: Type Mark',
    'Inside Door',
    'Inside door: URL',
    19,
    'Door - Panel',
    'Door - Frame',
    'Doors',
    'M_Single-Flush',
    'M_Single-Flush: Inside Door',
    'M_Single-Flush: Inside Door',
    2134,
    'Basic Wall: Outside wall',
    'M_Single-Flush: Inside Door',
    'M_Single-Flush: Inside Door',
    'Inside door: Fire Rating',
    '0',
    'Inside door: Fire Rating',
    3.7021,
    'Inside door: Manufacturer',
    'Doors',
    'Inside door: Type Mark',
    'Inside door: Keynote',
    'Inside door: Description',
    2134,
    915,
    'Inside door: URL',
    'Level: Level 1',
    'New Construction',
    3.18957899999998,
    0,
    2134,
    915,
    3.18957899999998,
    'Level: Level 1',
    0,
    'New Construction'
)

...

(
    'Schematic Type',
    '1/8 in Pilkington single glazing',
    3.6886,
    0.78,
    0.271105568508377,
    0.9,
    'By host',
    305,
    1830,
    915,
    19,
    3.17875400000013,
    0.0561146700000025,
    None,
    None,
    'Window: Assembly Code',
    'Window: Description',
    'Window: Keynote',
    'Window: Manufacturer',
    'Window: Model',
    '23.30.20.17.11',
    'Fixed Windows',
    'Window: Type Mark',
    '0915 x 1830mm',
    'Window: URL',
    38,
    'Sash',
    'Sash',
    'Glass',
    'Sash',
    'Windows',
    'M_Fixed',
    'Basic Wall: Outside wall',
    'M_Fixed: 0915 x 1830mm',
    'M_Fixed: 0915 x 1830mm',
    2135,
    'M_Fixed: 0915 x 1830mm',
    'M_Fixed: 0915 x 1830mm',
    'Window: Manufacturer',
    'Windows',
    '1',
    3.6886,
    'Window: Type Mark',
    'Window: Keynote',
    'Window: Description',
    1830,
    915,
    'Window: URL',
    'Level: Level 1',
    'New Construction',
    3.17875400000013,
    305.000000000004,
    1830,
    915,
    3.17875400000013,
    'Level: Level 1',
    305.000000000004,
    'New Construction'
)

(
    'Schematic Type',
    '1/8 in Pilkington single glazing',
    3.6886,
    0.78,
    0.271105568508377,
    0.9,
    'By host',
    305,
    1830,
    915,
    19,
    3.17875400000013,
    0.0561146700000025,
    None,
    None,
    'Window: Assembly Code',
    'Window: Description',
    'Window: Keynote',
    'Window: Manufacturer',
    'Window: Model',
    '23.30.20.17.11',
    'Fixed Windows',
    'Window: Type Mark',
    '0915 x 1830mm',
    'Window: URL',
    39,
    'Sash',
    'Sash',
    'Glass',
    'Sash',
    'Windows',
    'M_Fixed',
    'Basic Wall: Outside wall',
    'M_Fixed: 0915 x 1830mm',
    'M_Fixed: 0915 x 1830mm',
    2135,
    'M_Fixed: 0915 x 1830mm',
    'M_Fixed: 0915 x 1830mm',
    'Window: Manufacturer',
    'Windows',
    '1',
    3.6886,
    'Window: Type Mark',
    'Window: Keynote',
    'Window: Description',
    1830,
    915,
    'Window: URL',
    'Level: Level 1',
    'New Construction',
    3.17875400000013,
    305.000000000004,
    1830,
    915,
    3.17875400000013,
    'Level: Level 1',
    305.000000000004,
    'New Construction'
)

(
    'Schematic Type',
    '1/8 in Pilkington single glazing',
    3.6886,
    0.78,
    0.271105568508377,
    0.9,
    'By host',
    305,
    1830,
    915,
    19,
    3.17875400000013,
    0.0561146700000025,
    None,
    None,
    'Window: Assembly Code',
    'Window: Description',
    'Window: Keynote',
    'Window: Manufacturer',
    'Window: Model',
    '23.30.20.17.11',
    'Fixed Windows',
    'Window: Type Mark',
    '0915 x 1830mm',
    'Window: URL',
    40,
    'Sash',
    'Sash',
    'Glass',
    'Sash',
    'Windows',
    'M_Fixed',
    'Basic Wall: Outside wall',
    'M_Fixed: 0915 x 1830mm',
    'M_Fixed: 0915 x 1830mm',
    2135,
    'M_Fixed: 0915 x 1830mm',
    'M_Fixed: 0915 x 1830mm',
    'Window: Manufacturer',
    'Windows',
    '1',
    3.6886,
    'Window: Type Mark',
    'Window: Keynote',
    'Window: Description',
    1830,
    915,
    'Window: URL',
    'Level: Level 1',
    'New Construction',
    3.17875400000013,
    305.000000000004,
    1830,
    915,
    3.17875400000013,
    'Level: Level 1',
    305.000000000004,
    'New Construction'
)

(
    'Schematic Type',
    '1/8 in Pilkington single glazing',
    3.6886,
    0.78,
    0.271105568508377,
    0.9,
    'By host',
    305,
    1830,
    915,
    19,
    3.17875400000013,
    0.0561146700000025,
    None,
    None,
    'Window: Assembly Code',
    'Window: Description',
    'Window: Keynote',
    'Window: Manufacturer',
    'Window: Model',
    '23.30.20.17.11',
    'Fixed Windows',
    'Window: Type Mark',
    '0915 x 1830mm',
    'Window: URL',
    41,
    'Sash',
    'Sash',
    'Glass',
    'Sash',
    'Windows',
    'M_Fixed',
    'Basic Wall: Outside wall',
    'M_Fixed: 0915 x 1830mm',
    'M_Fixed: 0915 x 1830mm',
    2135,
    'M_Fixed: 0915 x 1830mm',
    'M_Fixed: 0915 x 1830mm',
    'Window: Manufacturer',
    'Windows',
    '1',
    3.6886,
    'Window: Type Mark',
    'Window: Keynote',
    'Window: Description',
    1830,
    915,
    'Window: URL',
    'Level: Level 1',
    'New Construction',
    3.17875400000013,
    305.000000000004,
    1830,
    915,
    3.17875400000013,
    'Level: Level 1',
    305.000000000004,
    'New Construction'
)

(
    'Schematic Type',
    '1/8 in Pilkington single glazing',
    3.6886,
    0.78,
    0.271105568508377,
    0.9,
    'By host',
    305,
    1830,
    915,
    19,
    3.17875400000013,
    0.0561146700000025,
    None,
    None,
    'Window: Assembly Code',
    'Window: Description',
    'Window: Keynote',
    'Window: Manufacturer',
    'Window: Model',
    '23.30.20.17.11',
    'Fixed Windows',
    'Window: Type Mark',
    '0915 x 1830mm',
    'Window: URL',
    42,
    'Sash',
    'Sash',
    'Glass',
    'Sash',
    'Windows',
    'M_Fixed',
    'Basic Wall: Outside wall',
    'M_Fixed: 0915 x 1830mm',
    'M_Fixed: 0915 x 1830mm',
    2135,
    'M_Fixed: 0915 x 1830mm',
    'M_Fixed: 0915 x 1830mm',
    'Window: Manufacturer',
    'Windows',
    '1',
    3.6886,
    'Window: Type Mark',
    'Window: Keynote',
    'Window: Description',
    1830,
    915,
    'Window: URL',
    'Level: Level 1',
    'New Construction',
    3.17875400000013,
    305.000000000004,
    1830,
    915,
    3.17875400000013,
    'Level: Level 1',
    305.000000000004,
    'New Construction'
)

## 2. LLM call: Question2Query

## Database Context

In [ ]:
# Function to get the structure of the database
def get_dB_schema(dB_path):
    conn = sqlite3.connect(dB_path)
    cursor = conn.cursor()

    schema_info = {}
    # Get a list of all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_names = cursor.fetchall()

    for table in table_names:
        table_name = table[0]
        column_names = []

        # Get the schema for the specific table
        cursor.execute(f"PRAGMA table_info({table_name});")
        schema = cursor.fetchall()

        for column in schema:
            column_names.append(column[1])

        schema_info[table_name] = column_names

    conn.close()
    return schema_info

In [ ]:
# Run an SQL query against the database
def execute_sql_query(dB_path, sql_query):
    # Connect to the SQLite database
    conn = sqlite3.connect(dB_path)
    cursor = conn.cursor()

    # Execute the SQL query
    cursor.execute(sql_query)
    result = cursor.fetchall()

    # Close the connection
    conn.close()

    return result

## LLM calls

In [ ]:

from openai import OpenAI
import random
from getpass import getpass

# Define OpenAI API access
OPENAI_API_KEY = getpass('Enter the secret value: ')
client = OpenAI(api_key=OPENAI_API_KEY)
completion_model = [
        {
            "model": "gpt-4o",
            "api_key": OPENAI_API_KEY,
            "cache_seed": random.randint(0, 100000),
        }]
completion_model = completion_model[0]['model']

Enter the secret value: ··········


In [ ]:
# Function to generate an SQL query out of a user question and an IFC database context
def generate_sql_query(dB_context: str, user_question: str) -> str:
    response = client.chat.completions.create(
        model=completion_model,
        messages=[
            {
                "role": "system",
                "content":
                       f"""
                You are a SQLite expert working with a database that has multiple tables, one for each building element type. \n
                Each table row represents an instance of a building element of that type. \n
                The schema of the database  is: \n {dB_context}. \n
                Pay special atenttion to the names of the tables and properties. Your query must use keywords that match perfectly.
                Follow these instructions:
                1. Given the user question, think about what data a query to the Database should fetch. Only data related to the question should be fetched.
                2. Output the SQL query to the Database. Do not use formatting characters, write only the query string.
                Provide only the SQL query in your response. Never output both. Do not use formatting characters like '```sql' or other extra text.
                Use only the table names and properties as shown in the schema.
                """
            },
            {
                "role": "user",
                "content": f"""
                # User question #
                {user_question}
                """,
            },
        ],
    )
    return response.choices[0].message.content

# Function to generate a natural reponsonse out of the retrieved IFC data
def generate_answer(sql_query: str, sql_result: str, user_question: str) -> str:
  response = client.chat.completions.create(
      model=completion_model,
      messages=[
        {
            "role": "system",
            "content":
                    f"""
                    You have to answer a user question according to the SQL query and its result.
                    Your goal is to answer in a concise and informative way, specifying the properties and tables that were relevant to create the answer.
                    If the sql query is empty or doesnt provide the information needed to answer the question, simply output: ´Given the provided IFC file, I cant answer that question.´
                    ### EXAMPLE ###
                    User Question: What is the area of the largest slab?
                    SQL Query: SELECT GlobalId, Dimensions_Area FROM IfcSlab ORDER BY Dimensions_Area DESC LIMIT 1;
                    SQL Result: [('3qq_RRlZrFqhCIHFKokT7x', 207.1385920365226)]
                    Answer: I looked at the Dimensions_Area property of IfcSlab elements and found that the area of the largest slab, with GlobalId '3qq_RRlZrFqhCIHFKokT7x', is 207.13 m².
            """,
        },
        {
            "role": "user",
            "content": f"""
            User question: {user_question}
            SQL Query: {sql_query}
            SQL Result: {sql_result}
            Answer:
            """,
        },
    ],
  )
  return response.choices[0].message.content

In [ ]:
# Define the question to ask the LLM
user_question = "How many total doors in the project"

# Get the schema of the database to give as context to the LLM
ifc_sql_db = "/content/ifcdatabase.db"
db_schema = get_dB_schema(ifc_sql_db)

# Ask the LLM to generate an SQL query for our question
sql_query = generate_sql_query(db_schema, user_question)
print(f"SQL Query: {sql_query}")

# Execute the SQL query against the database
sql_result = execute_sql_query(ifc_sql_db, sql_query)
print(f"SQL Result: {sql_result}")

# Ask the LLM to generate a natural response given the SQL result
llm_answer = generate_answer(sql_query, sql_result, user_question)
print(f"LLM Answer: {llm_answer}")

SQL Query: 
SELECT COUNT(*) FROM IfcDoor

SQL Result: [(18,)]

LLM Answer: I looked at the IfcDoor elements in the project and found that there are a total of 18 doors.

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
import gradio as gr
import sqlite3
from openai import OpenAI
import os

# Assuming these functions are defined elsewhere in your code

# Set up OpenAI client

# Path to your SQLite database
IFC_SQL_DB = "/content/ifcdatabase.db"

def query_ifc(user_question):
    # Get the schema of the database
    db_schema = get_dB_schema(IFC_SQL_DB)

    # Generate SQL query
    sql_query = generate_sql_query(db_schema, user_question)

    # Execute SQL query
    sql_result = execute_sql_query(IFC_SQL_DB, sql_query)

    # Generate natural language answer
    llm_answer = generate_answer(sql_query, sql_result, user_question)

    return f"SQL Query: {sql_query}\n\nSQL Result: {sql_result}\n\nAnswer: {llm_answer}"

# Create Gradio interface
iface = gr.Interface(
    fn=query_ifc,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question about the IFC model here..."),
    outputs="text",
    title="IFC Query System",
    description="Ask questions about your IFC model in natural language.",
    examples=[
        ["How many doors are in the project?"],
        ["What is the total area of all windows?"],
        ["How many floors does the building have?"],
        ["What is the height of the tallest wall?"]
    ]
)

# Launch the interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b5f276017b210a2559.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
